In [1]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import matplotlib.pyplot as plt

## Selection of covariates

After examining the graph we come to conclusion that there are no frontdoor paths from treatment to growth. All backdoor paths, but for the one that goes through unobserved confounders and dt are blocked because they contain colliders (e.g. dCoa is a collider). As unobserved confounders are not evaluated statistically, we do not have to adust for them. So, our only option for valid set is dt.

Logic is pretty the same when we consider the path from treatment to dANb. Only difference is that we can block the path treatment - uc - dt - growth - dANb by blocking either dt or Growth. So, {dt}, {growth} or {dt, growth} are valid sets for this case.

In [2]:
##Let's look at our data first
malocussion = pd.read_csv('malocclusion.csv')
malocussion

,dANB,dPPPM,dIMPA,dCoA,dGoPg,dCoGo,dT,Growth,Treatment
0,-3.2,-1.1,-4.2,1.0,4.0,3.7,5,0,0
1,-0.6,-0.5,3.8,2.6,-0.1,1.4,3,1,0
2,-1.6,-3.1,-6.0,4.3,4.2,7.1,5,0,0
3,-1.1,-2.1,-12.1,14.1,20.7,17.5,9,0,0
4,-1.1,0.0,-6.7,7.7,8.8,11.0,5,0,0
...,...,...,...,...,...,...,...,...,...
138,0.8,-2.1,-2.0,2.7,2.0,3.3,5,1,1
139,2.1,0.7,1.4,8.2,12.8,9.1,10,1,1
140,-0.2,-3.3,-2.7,6.8,3.4,10.9,4,1,1
141,1.5,-3.5,1.8,4.6,6.5,6.2,5,1,1


In [35]:
malocussion.describe()

,dANB,dPPPM,dIMPA,dCoA,dGoPg,dCoGo,dT,Growth,Treatment
count,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000
mean,-0.227273,-1.374825,-0.785315,5.987413,7.730769,6.732867,4.706294,0.405594,0.461538
std,1.826225,2.715046,5.080894,4.469692,5.532417,4.595141,2.550427,0.492733,0.500271
min,-5.100000,-9.300000,-19.000000,-0.900000,-1.400000,-2.600000,1.000000,0.000000,0.000000
25%,-1.350000,-2.750000,-3.450000,1.800000,3.200000,3.050000,3.000000,0.000000,0.000000
50%,-0.300000,-1.400000,-0.400000,5.500000,6.200000,6.300000,4.000000,0.000000,0.000000
75%,0.950000,0.050000,2.100000,9.750000,12.750000,10.350000,6.000000,1.000000,1.000000
max,4.900000,6.500000,12.000000,20.000000,23.300000,17.500000,12.000000,1.000000,1.000000


## treatment - growth

In [4]:
##Naive treatment-growth ate estimator
malocussion.Growth[malocussion.Treatment == 1].mean() - malocussion.Growth[malocussion.Treatment == 0].mean()

0.1471861471861472

In [32]:
##ols regression ate estimator
m = smf.ols('Growth ~ Treatment + dT', data=malocussion)
fitted = m.fit()
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                 Growth   R-squared:                       0.047
Model:                            OLS   Adj. R-squared:                  0.033
Method:                 Least Squares   F-statistic:                     3.433
Date:                Sun, 21 Apr 2024   Prob (F-statistic):             0.0350
Time:                        12:48:20   Log-Likelihood:                -97.770
No. Observations:                 143   AIC:                             201.5
Df Residuals:                     140   BIC:                             210.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4623      0.086      5.382      0.0

As we see, according to P>|t|, treatment seems to be an statistically significant variable. On the other hand, it is not very powerful, ar r^2 of our regression is close to 0. 

In [33]:
##Using causal inference library
from causalinference import CausalModel
adjustment_set = ['dT']

causal = CausalModel(
    Y=malocussion['Growth'].values, # outcome
    D=malocussion['Treatment'].values, # treatment
    X=malocussion[adjustment_set].values
)

In [34]:
causal.est_via_ols()
print(causal.estimates)


Treatment Effect Estimates: OLS

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE      0.202      0.087      2.338      0.019      0.033      0.372
           ATC      0.190      0.101      1.883      0.060     -0.008      0.387
           ATT      0.217      0.084      2.581      0.010      0.052      0.382



/Users/nikita/anaconda3/lib/python3.10/site-packages/causalinference/estimators/ols.py:21: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  olscoef = np.linalg.lstsq(Z, Y)[0]


In [36]:
##Using propensity score matching
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV

# classifier to estimate the propensity score
cls = LogisticRegression()

# calibration of the classifier
cls = CalibratedClassifierCV(cls)

X = malocussion[['Growth']]
y = malocussion['Treatment']
cls.fit(X, y)
malocussion['e'] = cls.predict_proba(X)[:,1].tolist()
malocussion.head()

,dANB,dPPPM,dIMPA,dCoA,dGoPg,dCoGo,dT,Growth,Treatment,e
0,-3.2,-1.1,-4.2,1.0,4.0,3.7,5,0,0,0.34318
1,-0.6,-0.5,3.8,2.6,-0.1,1.4,3,1,0,0.42124
2,-1.6,-3.1,-6.0,4.3,4.2,7.1,5,0,0,0.34318
3,-1.1,-2.1,-12.1,14.1,20.7,17.5,9,0,0,0.34318
4,-1.1,0.0,-6.7,7.7,8.8,11.0,5,0,0,0.34318


In [37]:
malocussion['w'] = malocussion['Treatment'] / malocussion['e'] + (1 - malocussion['Treatment']) / (1 - malocussion['e'])

In [38]:
m = smf.wls('Growth ~ Treatment + dT', data=malocussion, weights=malocussion['w'])
fitted = m.fit()
print(fitted.summary())

                            WLS Regression Results                            
Dep. Variable:                 Growth   R-squared:                       0.026
Model:                            WLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     1.858
Date:                Sun, 21 Apr 2024   Prob (F-statistic):              0.160
Time:                        19:12:56   Log-Likelihood:                -101.92
No. Observations:                 143   AIC:                             209.8
Df Residuals:                     140   BIC:                             218.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4879      0.094      5.175      0.0

With this regression we see that treatment effect is neither powerful nor statistically significany

## treatment - dANb

In [15]:
##Naive treatment-growth ate estimator
malocussion.dANB[malocussion.Treatment == 1].mean() - malocussion.dANB[malocussion.Treatment == 0].mean()

2.0287878787878784

In [18]:
##ols regression ate estimator
m = smf.ols('dANB ~ Treatment + Growth', data=malocussion)
fitted = m.fit()
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                   dANB   R-squared:                       0.407
Model:                            OLS   Adj. R-squared:                  0.398
Method:                 Least Squares   F-statistic:                     48.04
Date:                Sun, 21 Apr 2024   Prob (F-statistic):           1.31e-16
Time:                        12:37:24   Log-Likelihood:                -251.17
No. Observations:                 143   AIC:                             508.3
Df Residuals:                     140   BIC:                             517.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.5600      0.181     -8.609      0.0

As we see, both treatment and growth are statistically significant variables (and treatment coefficient is even slightly bigger). R^2 of 0.4 shows decent probabilistic abilities of our model as well

In [30]:
##Using causal inference library
adjustment_set = ['Growth']

causal = CausalModel(
    Y=malocussion['dANB'].values, # outcome
    D=malocussion['Treatment'].values, # treatment
    X=malocussion[adjustment_set].values
)

In [31]:
causal.est_via_ols()
print(causal.estimates)


Treatment Effect Estimates: OLS

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE      1.856      0.240      7.743      0.000      1.386      2.326
           ATC      1.860      0.245      7.580      0.000      1.379      2.341
           ATT      1.852      0.239      7.760      0.000      1.384      2.320



/Users/nikita/anaconda3/lib/python3.10/site-packages/causalinference/estimators/ols.py:21: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  olscoef = np.linalg.lstsq(Z, Y)[0]


In [40]:
causal.est_via_matching(weights='maha', bias_adj=True)
print(causal.estimates)


Treatment Effect Estimates: OLS

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE      0.202      0.087      2.338      0.019      0.033      0.372
           ATC      0.190      0.101      1.883      0.060     -0.008      0.387
           ATT      0.217      0.084      2.581      0.010      0.052      0.382

Treatment Effect Estimates: Matching

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE      0.320      0.120      2.678      0.007      0.086      0.554
           ATC      0.338      0.155      2.179      0.029      0.034      0.641
           ATT      0.300      0.130      2.307      0.021      0.045      0.554



/Users/nikita/anaconda3/lib/python3.10/site-packages/causalinference/estimators/matching.py:100: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  return np.linalg.lstsq(X, Y)[0][1:]  # don't need intercept coef


We see that other ways of estitmations give similar results